In [1]:
import numpy as np
from GO import GO
from PPI import PPI
from PathManager import PathManager as PM
from ParamHandler import ParamHandler as PH

In [2]:
ph = PH()
pm = PM(ph)
go = GO(pm, ph)
ppi = PPI(pm, ph)

(19576, 19576) 19000 576


In [3]:
nn = np.random.uniform(0, 19576, size=19576*1000).astype(np.uint16)
np.random.shuffle(nn)
nn = np.reshape(nn, (19576, 1000))

In [4]:
nn, nn.shape, nn.min()

(array([[18562, 16696, 15368, ...,   337, 12855,  5574],
        [16853,  2313,  2775, ...,  6885, 15580, 15922],
        [13435, 15032,  3325, ...,  5664,  1749,  9702],
        ...,
        [ 3622,  5243, 15104, ...,  3991,  2978,  1647],
        [ 2280, 16676, 12206, ...,  6063, 15681, 17273],
        [ 3131, 10145,  1449, ..., 13775, 11423, 13556]], dtype=uint16),
 (19576, 1000),
 0)

In [5]:
weights = 1.0/(np.arange(1000)+1)
weights *= 1.0/(weights.sum()+1.0/1000)
weights.shape

(1000,)

In [6]:
weights = weights.astype(np.float64)
go.data = go.data.astype(np.float64)

In [7]:
weights.dtype, go.data.dtype

(dtype('float64'), dtype('float64'))

In [8]:
labeled = go.data[nn[177]]
labeled.shape

(1000, 16322)

In [9]:
%timeit go.data[nn[177]]

41.9 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit np.dot(weights, labeled)

4.98 ms ± 38 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit np.dot(weights, go.data[nn[177]])

47.6 ms ± 670 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit np.asarray([np.dot(weights, go.data[n]) for n in nn[:200]])

9.71 s ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
go.data = go.data.astype(np.float32)
weights = weights.astype(np.float32)

In [14]:
weights.dtype, go.data.dtype

(dtype('float32'), dtype('float32'))

In [15]:
%timeit [go.data[nn[i]] for i in range(200)]

4.21 s ± 178 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit np.dot(weights, labeled)

4.94 ms ± 63.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit np.dot(weights, go.data[nn[177]])

25.3 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
import numba
N=200
ss = np.empty([N,go.data.shape[1]],dtype=np.float32)
dg = (go.data.T)+np.float32(0.0)
@numba.njit(parallel=True,fastmath=True)
def foo (N,nn,w,gd,ss):
    
    for i in range(N):
        ss[i] = np.dot(gd[:,nn[i]],w)
%timeit (foo(200,nn,weights,dg,ss))       
@numba.njit(parallel=True,fastmath=True)
def foo2 (N,nn,w,gd,ss):
    sm = np.empty(gd.shape[1],dtype=np.float32)
    for i in range(N):
        
        tmp = gd[nn[i]]
       # print (tmp.shape,type(tmp),type(tmp[0,0]))
        for j in numba.prange(w.shape[0]):
            sm+= w[j]*tmp[j]
        ss[i] = sm.copy()       
    
%timeit (foo2(200,nn,weights,go.data,ss))

8.78 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.47 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:

@numba.jit()
def foo (N,nn,w,gd):
    ss = np.empty([N,gd.shape[1]],dtype=np.float32)
    for i in range(N):
        ss[i] = np.dot(w,gd[nn[i]])
    return ss
%timeit (foo(200,nn,weights,go.data))

4.54 s ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit np.asarray([np.dot(weights, go.data[n]) for n in nn[:200]])

5.05 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
preds = np.zeros(shape=(ppi.data.shape[0], go.data.shape[1]), dtype=np.float32)
def predict():
    for i in range(200):
        preds[i]+= np.dot(weights, go.data[nn[i]])
%timeit predict()

5.04 s ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
preds = np.zeros(shape=(ppi.data.shape[0], go.data.shape[1]), dtype=np.float32)
def predict():
    for i in range(200):
        preds[i] = np.dot(weights, go.data[nn[i]])
%timeit predict()

5.05 s ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit np.asarray([np.dot(weights, go.data[n]) for n in nn[:200]])

4.93 s ± 60.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit np.asarray([np.dot(weights, go.data[n]) for n in nn])

7min 45s ± 385 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(preds.dtype, weights.dtype, go.data.dtype)